<a href="https://colab.research.google.com/github/IA-Programming/phixtral-inference-into-a-free-colab-8bit/blob/main/phixtral-2x2_8bit-with-gradio-interface-in-free-google-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of the libraries

In [3]:
!pip install -qU torch>=2.0 transformers>=4.35.0 gradio>=4.13.0 einops accelerate bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.13 requires torch<2.2,>=1.10, but you have torch 2.2.0 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.


In [2]:
!pip install --quiet --upgrade gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


# Installation of the model

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

torch.set_default_device("cuda")

# Loading the tokenizer and model from Hugging Face's model hub.
tokenizer = AutoTokenizer.from_pretrained(
    "mlabonne/phixtral-2x2_8",
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "mlabonne/phixtral-2x2_8",
    torch_dtype="auto",
    load_in_8bit=True,
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/mlabonne/phixtral-2x2_8:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/34.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mlabonne/phixtral-2x2_8:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00001.safetensors:   0%|          | 0.00/8.92G [00:00<?, ?B/s]

# Classes to use the model

In [2]:
# Defining a custom stopping criteria class for the model's text generation.
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50256, 50295]  # IDs of tokens where the generation should stop.
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:  # Checking if the last generated token is a stop token.
                return True
        return False


# Function to generate model predictions.
def predict(message, history):
    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    # Formatting the input for the model.
    system_prompt = "<|im_start|>system\nYou are Phixtral, a helpful AI assistant.<|im_end|>"
    messages = system_prompt + "".join(["".join(["\n<|im_start|>user\n" + item[0], "<|im_end|>\n<|im_start|>assistant\n" + item[1]]) for item in history_transformer_format])
    input_ids = tokenizer([messages], return_tensors="pt").to('cuda')
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '<|im_end|>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
        yield partial_message

# Running the model in a gradio app

In [3]:
import gradio as gr
# Setting up the Gradio chat interface.
gr.ChatInterface(predict,
                 description="""
                 <center><img src="https://i.imgur.com/CJSeIGg.png" width="33%"></center>\n\n
                 Chat with [mlabonne/phixtral-2x2_8](https://huggingface.co/mlabonne/phixtral-2x2_8), the first Mixture of Experts made by merging two fine-tuned [microsoft/phi-2](https://huggingface.co/microsoft/phi-2) models.
                 This small model (4.46B param) is good for various tasks, such as programming, dialogues, story writing, and more.\n\n
                 ❤️ If you like this work, please follow me on [Hugging Face](https://huggingface.co/mlabonne) and [Twitter](https://twitter.com/maximelabonne).
                 """,
                 examples=[
                     'Can you solve the equation 2x + 3 = 11 for x?',
                     'Write an epic poem about Ancient Rome.',
                     'Who was the first person to walk on the Moon?',
                     'Use a list comprehension to create a list of squares for numbers from 1 to 10.',
                     'Recommend some popular science fiction books.',
                     'Can you write a short story about a time-traveling detective?'
                 ],
                 theme=gr.themes.Soft(primary_hue="orange"),
                 ).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b1354b8bf9f507f171.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# References

| Notebook Name | Description | Link |
| --- | --- | --- |
| [phixtral-4x2_8](https://huggingface.co/mlabonne/phixtral-4x2_8) | Huggingface model card and the huggingface space to prove the model works | [![Hugging Face](https://img.shields.io/badge/Open%20In-Hugging%20Face-blue.svg)](https://huggingface.co/spaces/mlabonne/phixtral-chat) |
| [**Phixtral 4x2_8B: Efficient Mixture of Experts with phi-2 models WOW**](https://colab.research.google.com/drive/1k6C_oJfEKUq0mtuWKisvoeMHxTcIxWRa?usp=sharing) | This video introduces the Phixtral Mixture of Experts (PMoE) model. | [![](https://img.shields.io/badge/Open%20in%20YouTube-FF0000?logo=youtube&style=for-the-badge)](https://www.youtube.com/watch?v=uE2CrVPewV4) |